In [8]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# Define the Pacejka Magic Formula for lateral force (FY) with five coefficients
def pacejka_model(alpha, B, C, D, E, F):
    return D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F)

# Function to process a single tire dataset
def process_tire_data(file_path):
    # Load the data
    metric_data = pd.read_csv(file_path)

    # Iterate over columns and modify column names
    for col in metric_data.columns:
        metric_data.rename(columns={col: f"{col} {metric_data[col][0]}"}, inplace=True)

    # Drop the first row since it's now serving as part of the headers
    metric_data = metric_data.drop(0)

    # Reset index
    metric_data.reset_index(drop=True, inplace=True)

    # Convert all values to floats
    metric_data = metric_data.apply(pd.to_numeric, errors='coerce')

    # Drop any rows with NaN values (if necessary)
    metric_data = metric_data.dropna()

    # Reset index
    metric_data.reset_index(drop=True, inplace=True)

    # Define your bin edges based on the data range or desired intervals
    pressure_bins = np.unique(np.round(metric_data['P kPa']))
    camber_bins = np.unique(np.round(metric_data['IA deg']))

    # Assuming load was varied in steps of 50 N
    load_step = 50
    min_load = int(metric_data['FZ N'].min() // load_step * load_step)
    max_load = int(metric_data['FZ N'].max() // load_step * load_step)
    load_bins = np.arange(min_load, max_load + load_step, load_step)

    # Initialize empty lists to store binned data
    binned_data = []

    # Loop through each bin combination
    for pressure in pressure_bins:
        for camber in camber_bins:
            for load in load_bins:
                # Filter data for current bin
                bin_filter = (np.round(metric_data['P kPa']) == pressure) & \
                             (np.round(metric_data['IA deg']) == camber) & \
                             (metric_data['FZ N'] >= load) & (metric_data['FZ N'] < load + load_step)
                filtered_data = metric_data[bin_filter]

                if not filtered_data.empty:
                    # Calculate mean for 'SA deg' and 'FY N' in the current bin
                    mean_SA = filtered_data['SA deg'].mean()
                    mean_FY = filtered_data['FY N'].mean()

                    # Store the mean values along with the bin identifiers
                    binned_data.append([pressure, camber, load, mean_SA, mean_FY])

    # Convert binned data into a DataFrame
    binned_df = pd.DataFrame(binned_data, columns=['Pressure', 'Camber', 'Load', 'Mean SA', 'Mean FY'])

    # Fit the Pacejka model to the binned data
    initial_guess = [0.1, 0.1, 100, 0.1, 0]  # Update as needed based on your knowledge of the data
    bounds = (0, np.inf)  # Coefficients are non-negative

    # Ensure there's enough data to fit
    if not binned_df.empty and len(binned_df) > len(initial_guess):
        params_optimal, _ = curve_fit(pacejka_model, binned_df['Mean SA'], binned_df['Mean FY'],
                                      p0=initial_guess, maxfev=150000)
    else:
        raise ValueError("Not enough data to fit the model or binned data is empty.")

    # Return the fitted parameters and the binned data for plotting
    return params_optimal, binned_df


In [14]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import os

# Assuming the function process_tire_data is already defined and returns fitted params and binned data

def fitted_curve(alpha_range, B, C, D, E, F):
    return pacejka_model(alpha_range, B, C, D, E, F)

# List of file paths for different datasets
file_paths = [
    '/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run50.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run51.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run52.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run53.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run54.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run124.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run125.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run126.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run127.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run128.csv','/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run129.csv',
    # ... add as many dataset file paths as needed
]

# Dictionary to store results for each dataset
tire_data_results = {}

# Process each dataset file and store the results
for file_path in file_paths:
    params_optimal, binned_df = process_tire_data(file_path)
    tire_data_results[file_path] = {'params': params_optimal, 'binned_df': binned_df}

# Plot all the fitted curves using Plotly
# Initialize Plotly figure
fig = go.Figure()

# Assuming tire_data_results is populated as before
for file_path, data in tire_data_results.items():
    # Generate a range of slip angles for plotting
    alpha_range = np.linspace(data['binned_df']['Mean SA'].min(), data['binned_df']['Mean SA'].max(), num=500)

    # Calculate the fitted curve over the range of alpha
    fy_fitted = fitted_curve(alpha_range, *data['params'])

    # Extract the run number from the file path for the legend title
    run_name = os.path.splitext(os.path.basename(file_path))[0]  # This will extract 'run50' from 'run50.csv'

    # Add the fitted curve to the plot with the run name as the legend title
    fig.add_trace(go.Scatter(x=alpha_range, y=fy_fitted, mode='lines', name=run_name))

# Update plot layout
fig.update_layout(
    title='Comparison of Fitted Tire Curves',
    xaxis_title='Slip Angle (SA deg)',
    yaxis_title='Lateral Force (FY N)',
    legend_title='Dataset'
)

# Show plot
#fig.show()


/tmp/ipykernel_610/422446482.py:12: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.



In [15]:
import os
import plotly.graph_objects as go
import plotly.express as px

# Initialize Plotly figure for the raw data comparison
fig_raw = go.Figure()

# Assuming tire_data_results is populated as before
for file_path, data in tire_data_results.items():
    # Extract the run number from the file path for the legend title
    run_name = os.path.splitext(os.path.basename(file_path))[0]  # This will extract 'run50' from 'run50.csv'

    # Add the raw data to the plot with the run name as the legend title
    fig_raw.add_trace(go.Scatter(x=data['binned_df']['Mean SA'], y=data['binned_df']['Mean FY'], mode='markers', name=run_name))

# Update plot layout for the raw data comparison
fig_raw.update_layout(
    title='Comparison of Raw Tire Data',
    xaxis_title='Slip Angle (SA deg)',
    yaxis_title='Lateral Force (FY N)',
    legend_title='Dataset'
)

# Show the raw data comparison plot
fig_raw.show()

# Initialize Plotly figure for the fitted curves comparison
fig_fitted = go.Figure()

# Use the same colors for each run in both plots
color_palette = px.colors.qualitative.Plotly

# Plot raw data and fitted curves
for index, (file_path, data) in enumerate(tire_data_results.items()):
    # Choose color
    color = color_palette[index % len(color_palette)]

    # Extract the run number from the file path for the legend title
    run_name = os.path.splitext(os.path.basename(file_path))[0]

    # Generate a range of slip angles for plotting
    alpha_range = np.linspace(data['binned_df']['Mean SA'].min(), data['binned_df']['Mean SA'].max(), num=500)

    # Calculate the fitted curve over the range of alpha
    fy_fitted = fitted_curve(alpha_range, *data['params'])

    # Add the raw data to the plot
    #fig_fitted.add_trace(go.Scatter(x=data['binned_df']['Mean SA'], y=data['binned_df']['Mean FY'], mode='markers', name=run_name, marker_color=color))

    # Add the fitted curve to the plot
    fig_fitted.add_trace(go.Scatter(x=alpha_range, y=fy_fitted, mode='lines', name=f'Fitted {run_name}', line=dict(color=color)))

# Update plot layout for the fitted curves comparison
fig_fitted.update_layout(
    title='Comparison of Fitted Tire Curves',
    xaxis_title='Slip Angle (SA deg)',
    yaxis_title='Lateral Force (FY N)',
    legend_title='Dataset'
)

# Show the fitted curves comparison plot
fig_fitted.show()
